In [4]:
import pandas as pd

In [6]:
df = pd.read_csv('Statcast_2020.csv')
df19 = pd.read_csv('Statcast_2019.csv')

df_merge = df.append(df19)
df_clean = df_merge[['pitch_type','release_speed','release_pos_x','release_pos_z','player_name','zone','game_type','stand','p_throws','type','balls','strikes','pfx_x','pfx_z','plate_x','plate_z','inning','vx0','vy0','ax','ay','az','sz_top','sz_bot','effective_speed','release_spin_rate','release_extension','release_pos_y','at_bat_number','pitch_number','pitch_name','spin_axis','delta_home_win_exp','delta_run_exp']]
df_clean.head()

,pitch_type,release_speed,release_pos_x,release_pos_z,player_name,zone,game_type,stand,p_throws,type,...,effective_speed,release_spin_rate,release_extension,release_pos_y,at_bat_number,pitch_number,pitch_name,spin_axis,delta_home_win_exp,delta_run_exp
0,FF,98.8,-0.41,6.46,"Fairbanks, Pete",14.0,F,L,R,X,...,100.2,2483.0,7.2,53.26,64,2,4-Seam Fastball,198.0,0.034,-0.370
1,FF,98.7,-0.33,6.62,"Fairbanks, Pete",12.0,F,L,R,B,...,100.0,2522.0,7.2,53.32,64,1,4-Seam Fastball,189.0,0.003,0.040
2,SL,89.6,-0.10,6.79,"Fairbanks, Pete",5.0,F,R,R,S,...,90.8,2537.0,6.9,53.56,63,6,Slider,58.0,0.053,-0.392
3,FF,100.4,-0.38,6.50,"Fairbanks, Pete",6.0,F,R,R,S,...,101.7,2469.0,7.1,53.37,63,5,4-Seam Fastball,191.0,0.000,-0.102
4,FF,97.6,-0.18,6.63,"Fairbanks, Pete",12.0,F,R,R,S,...,98.9,2339.0,7.1,53.42,63,4,4-Seam Fastball,191.0,0.000,-0.053


In [9]:
df_cole = df_clean[df_clean['player_name'] == 'Cole, Gerrit']
df_degrom = df_clean[df_clean['player_name'] == 'DeGrom, Jacob']
df_scherzer = df_clean[df_clean['player_name'] == 'Scherzer, Max']
df_bauer = df_clean[df_clean['player_name'] == 'Bauer, Trevor']
df_bieber = df_clean[df_clean['player_name'] == 'Bieber, Shane']
df_kershaw = df_clean[df_clean['player_name'] == 'Kershaw, Clayton']
df_lamet = df_clean[df_clean['player_name'] == 'Lamet, Dinelson']
df_nola = df_clean[df_clean['player_name'] == 'Nola, Aaron']
df_buehler = df_clean[df_clean['player_name'] == 'Buehler, Walker']


In [7]:
df_r = df_clean[df_clean['p_throws'] == 'R']
df_l = df_clean[df_clean['p_throws'] == 'L']

In [10]:
file_path = ('resources')
df_cole.to_csv('resources/cole.csv')
df_degrom.to_csv('resources/degrom.csv')
df_scherzer.to_csv('resources/scherzer.csv')
df_bauer.to_csv('resources/bauer.csv')
df_bieber.to_csv('resources/bieber.csv')
df_kershaw.to_csv('resources/kershaw.csv')
df_lamet.to_csv('resources/lamet.csv')
df_nola.to_csv('resources/nola.csv')
df_buehler.to_csv('resources/buehler.csv')


In [8]:
df_l.to_csv('resources/l.csv')
df_r.to_csv('resources/r.csv')